In [1]:
from deepdiff import DeepDiff

In [2]:
import os, io, re

In [3]:
from craftisan.tool_db.base import Session, Tool, Pocket

In [4]:
NO_TOOL = {
    'T': 0,
    'C': 0.0,
    'D': 0.0,
    'P': 0,
    'X': 0.0,
    'Y': 0.0,
    'Z': 0.0,
    'R': 'No Tool Loaded',
}

In [5]:
def getTTData(file, columns='TPXYZCDR'):
    if not os.path.exists(file):
        print(f"Tool table file does not exist: {file}")
        return {}
    
    with io.open(file, 'r') as fh:
        lines = [line.strip() for line in fh.readlines()]

    table = {0: NO_TOOL,}
    for line in lines:

        data, sep, comment = line.partition(';')
        items = re.findall(r"([A-Z]+[0-9.+-]+)", data.replace(' ', ''))

        tool = NO_TOOL.copy()
        for item in items:
            descriptor = item[0]
            if descriptor in columns:
                value = item[1:]
                if descriptor in ('T', 'P', 'Q'):
                    try:
                        tool[descriptor] = int(value)
                    except:
                        print(f'Error converting value to int: {value}')
                        break
                else:
                    try:
                        tool[descriptor] = float(value)
                    except:
                        print(f'Error converting value to float: {value}')
                        break

        tool['R'] = comment.strip()

        tnum = tool['T']
        if tnum == -1:
            continue

        table[tnum] = tool

    return table.copy()

In [6]:
def loadToolTable():
    with Session() as session:
        pockets = session.query(Pocket).where(Pocket.toolId.is_not(None)).all()

        table = {0: NO_TOOL}
        table.update({
            p.tool.id: p.tool.toTableDict() 
            for p in pockets if p.tool is not None
        })

    return table.copy()

In [7]:
def merge(source, destination):
    """
    run me with nosetests --with-doctest file.py

    >>> a = { 'first' : { 'all_rows' : { 'pass' : 'dog', 'number' : '1' } } }
    >>> b = { 'first' : { 'all_rows' : { 'fail' : 'cat', 'number' : '5' } } }
    >>> merge(b, a) == { 'first' : { 'all_rows' : { 'pass' : 'dog', 'fail' : 'cat', 'number' : '5' } } }
    True
    """
    for key, value in source.items():
        if isinstance(value, dict):
            # get node or create one
            node = destination.setdefault(key, {})
            merge(value, node)
        else:
            destination[key] = value

    return destination

In [8]:
def merge_dicts(dict1, dict2):
    # Find differences between the dictionaries
    diff = DeepDiff(dict1, dict2)

    # Apply differences to the first dictionary to merge them
    for path, change in diff.items():
        if path[0] == 'dictionary_item_removed':
            key = path[1]
            dict1.pop(key, None)
        elif path[0] == 'dictionary_item_added' or path[0] == 'values_changed':
            key = path[1][0]
            dict1[key] = change['new_value']

    return dict1

In [9]:
def merge_and_get_intersection(dict1, dict2):
    # Find differences between the dictionaries
    diff = DeepDiff(dict1, dict2)

    # Create a new dictionary with only the common elements
    intersection_dict = {}
    for path, change in diff.items():
        if path[0] == 'values_changed':
            key = path[1][0]
            intersection_dict[key] = dict1[key]

    return intersection_dict

In [10]:
def merge_and_get_changed(dict1, dict2):
    # Find differences between the dictionaries
    diff = DeepDiff(dict1, dict2)

    # Create a new dictionary with only the changed elements
    changed_dict = {}
    for path, change in diff.items():
        if path[0] == 'values_changed':
            key = path[1][0]
            changed_dict[key] = dict2[key]
        elif path[0] == 'dictionary_item_added':
            key = path[1][0]
            changed_dict[key] = dict2[key]

    return changed_dict

In [11]:
tc = loadToolTable()

In [12]:
tt = getTTData("/home/cnc/linuxcnc/configs/craftisan_h200/tool.tbl")

In [9]:
diff = DeepDiff(tt, tc)

In [ ]:
diff

In [11]:
tt == tc

False

In [14]:
new_tt = merge_dicts(tt.copy(), tc)

In [22]:
new_tt

{0: {'T': 0,
  'C': 0.0,
  'D': 0.0,
  'P': 0,
  'X': 0.0,
  'Y': 0.0,
  'Z': 0.0,
  'R': 'No Tool Loaded'},
 1000: {'T': 1000,
  'C': 0.0,
  'D': 6.0,
  'P': 308,
  'X': 0.0,
  'Y': 0.0,
  'Z': 167.682,
  'R': 'borgia d6 + faska wz45mm'},
 1001: {'T': 1001,
  'C': 0.0,
  'D': 8.0,
  'P': 307,
  'X': 0.0,
  'Y': 0.0,
  'Z': 124.05,
  'R': 'trough drill d8'},
 1002: {'T': 1002,
  'C': 0.0,
  'D': 8.0,
  'P': 312,
  'X': 0.0,
  'Y': 0.0,
  'Z': 123.504,
  'R': 'blind drill d8 i47'},
 1501: {'T': 1501,
  'C': 0.0,
  'D': 12.0,
  'P': 315,
  'X': 0.0,
  'Y': 0.0,
  'Z': 97.423,
  'R': 'Фрезер MDF D12.0 Z12+1'},
 1502: {'T': 1502,
  'C': 0.0,
  'D': 6.0,
  'P': 316,
  'X': 0.0,
  'Y': 0.0,
  'Z': 119.25,
  'R': 'd6 z3 mdf'},
 1801: {'T': 1801,
  'C': 0.0,
  'D': 7.5,
  'P': 310,
  'X': 0.0,
  'Y': 0.0,
  'Z': 125.32,
  'R': 'd7.5 z2'},
 2201: {'T': 2201,
  'C': 0.0,
  'D': 19.7,
  'P': 301,
  'X': 0.0,
  'Y': 0.0,
  'Z': 199.588,
  'R': '20mm z3'},
 2202: {'T': 2202,
  'C': 0.0,
  'D': 10.0

In [26]:
new_tt == tc

False

In [20]:
tt2 = merge_and_get_intersection(tt, tc)

In [21]:
tt2

{}

In [24]:
tt3 = merge_and_get_changed(tt, tc)

In [25]:
tt3

{}

In [29]:
DeepDiff(new_tt, tc).values()

dict_values([{"root[1001]['R']": {'new_value': 'Проходна бургия D8.0 I46.0 LT124.05', 'old_value': 'trough drill d8'}, "root[2206]['Z']": {'new_value': 161.317, 'old_value': 158.317}, "root[2501]['R']": {'new_value': 'V-Канал A91* D51.0', 'old_value': 'v-kanal d51'}, "root[5101]['R']": {'new_value': 'Гарга T5.89 D178 Z16', 'old_value': 'garga t6 d180 z16'}, "root[4101]['R']": {'new_value': 'Двоен радиус R3/R5 TH20 LT116.105', 'old_value': 'double r3 th20 LT115.1'}, "root[9501]['P']": {'new_value': 408, 'old_value': 404}, "root[2502]['R']": {'new_value': 'V-Канал A120* D87.5', 'old_value': 'v-kanal 120*'}, "root[2504]['R']": {'new_value': 'V-Канал A150* D96.5', 'old_value': 'V-Groover d96 a150*'}, "root[1002]['R']": {'new_value': 'Глуха бургия D8.0 I47.0', 'old_value': 'blind drill d8 i47'}, "root[5102]['R']": {'new_value': 'Фрезова глава CMT D62.0 H80.0', 'old_value': 'Frezova glava CMT d62 H80'}, "root[9501]['R']": {'new_value': 'Агрегат за брави D15.5 LT136.12 LA-16.0', 'old_value': 

In [30]:
tt4 = merge(tt, tc)

In [31]:
DeepDiff(tc, tt4)

{}

In [ ]:
tt4

In [13]:
diff = DeepDiff(tt, tc, exclude_regex_paths="\['R'\]")

In [15]:
diff2 = DeepDiff(tt, tc)

In [17]:
diff

{}

In [16]:
diff2

{'values_changed': {"root[2207]['R']": {'new_value': 'Груб фрезер D19.8 Z3',
   'old_value': 'rough d20 z3'},
  "root[9501]['R']": {'new_value': 'Агрегат за брави D15.5 LT136.12 LA-16.0',
   'old_value': 'Агрегат за брави D15.5 LT136.12 LA-16.'}}}

In [18]:
comment = "Агрегат за брави D15.5 LT136.12 LA-16.0 noiwjdoiwjdoi"

In [20]:
f'; {comment:.37}'

'; Агрегат за брави D15.5 LT136.12 LA-16.0 noiwjdoiwjdoi'

In [1]:
dim = {k: 0 for k in 'lhs'}
ofs = {k: 0 for k in 'xyz'}

pice_data = {**dim, **ofs}
pice_data

{'l': 0, 'h': 0, 's': 0, 'x': 0, 'y': 0, 'z': 0}

In [3]:
{p: dim[k] for p, k in zip('xyz', 'lhs')}

{'x': 0, 'y': 0, 'z': 0}

In [4]:
{p: pice_data[k] for p, k in zip('xyzijk', 'lhsxyz')}

{'x': 0, 'y': 0, 'z': 0, 'i': 0, 'j': 0, 'k': 0}

In [6]:
merged = {p: pice_data[k] for p, k in zip('xyzijk', 'lhsxyz')}
gcode = ['G45']
gcode.extend([f'{k.upper()}{merged[k]:.3f}' for k in 'xyzijk'])

' '.join(gcode)

'G45 X0.000 Y0.000 Z0.000 I0.000 J0.000 K0.000'

In [3]:
def test_func(x):
    if x is None or x == 0:
        return (True, x)

    return (False, x)

In [6]:
row_data = [128, 0, 7, 14, 9, None, None]
# tuple(None if x == 0 else x for x in row_data)
# tuple(None if x == 0 else x for x in row_data)

(128, None, 7, 14, 9, None, None)

In [11]:
row_data = [128, 0, 7, 14, 9, None, None]
row_data = tuple(None if x == 0 else x for x in row_data)
# tuple(sorted(row_data[1:], key=test_func))
tuple(sorted(row_data[1:], key=lambda x: (x is not None, x), reverse=True))

(14, 9, 7, None, None, None)